# Microsoft (MSFT) ¿En qué meses alcanzó precios más altos y mas bajos?

### 1. Instalación de librerias

*   yfinance → para traer datos de Yahoo Finance
*   plotly → para graficar los datos de manera interactiva

In [2]:
!pip install yfinance plotly pandas

### 2. Importación de librerías

In [41]:
import yfinance as yf
import pandas as pd
import plotly.express as px

### 3. Descargamos datos de Microsoft (MSFT)

Ajustamos fechas el rango a los ultimos meses del año y descargamos datos de cierre diario de MSFT con la variable (data).

1.   **Download→** descarga los datos históricos de un ticker.

2.   **Parámetros:**

*   "MSFT" → ticker de Microsoft.
*   start y end → rango de fechas.

3.   **["Close"]→** precio de cierre diario.

Resultado: un Series de Pandas (una sola columna con índice de fechas).


In [73]:
data = yf.download("MSFT", start="2025-01-01", end="2025-08-31")["Close"]


/tmp/ipython-input-1030420102.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("MSFT", start="2025-01-01", end="2025-08-31")["Close"]
[*********************100%***********************]  1 of 1 completed


### 4. Agrupamos por mes (promedio, máximo y mínimo de cada uno)

*   **.resample("M")→**
Agrupa los datos por un intervalo de tiempo.
"M" significa fin de mes.
Ejemplo: todos los precios de enero se agrupan juntos, luego febrero, etc.

*   **.agg(["max", "min"])→**
Aplica funciones de agregación sobre cada grupo.
"max" → el precio máximo del mes.
"min" → el precio mínimo del mes.
Devuelve un DataFrame con dos columnas: max y min.

In [67]:
monthly = data.resample("M").agg(["max", "min"])

/tmp/ipython-input-1726459881.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = data.resample("M").agg(["max", "min"])


### 5. Identificamos el mes con el precio más alto y el más bajo

*   **monthly['MSFT_max'] →** selecciona la columna que contiene los valores máximos de MSFT en cada mes.

*   **.idxmax() →** devuelve el índice (fecha) donde ese valor máximo ocurrió.

In [75]:
max_month = monthly['MSFT_max'].idxmax()
min_month = monthly['MSFT_min'].idxmin()

print(f"Mes con el precio más alto: {max_month.strftime('%B %Y')}")
print(f"Mes con el precio más bajo: {min_month.strftime('%B %Y')}")

Mes con el precio más alto: August 2025
Mes con el precio más bajo: January 2023


### 6. Visualización con Plotly

Se crea un gráfico de líneas que muestra la evolución de los precios máximos y mínimos mensuales de MSFT en un gráfico interactivo.

*   **monthly →** DataFrame con los precios máximos y mínimos por mes.

*   x=monthly.index → eje X = fechas (meses).

*   y=["MSFT_max","MSFT_min"] → eje Y = dos líneas (máximos y mínimos).

*   **title →** título del gráfico.

*   **labels →** renombra ejes y leyenda para que se entiendan mejor.

*   **fig.show()→** Muestra el gráfico interactivo en Colab (puedes mover, hacer zoom, ver valores).


In [77]:
fig = px.line(
    monthly,
    x=monthly.index,
    y=["MSFT_max","MSFT_min"],
    title="Precios mensuales máximos y mínimos de Microsoft (MSFT)",
    labels={"value": "Precio USD", "index": "Fecha", "variable": "Tipo"}
)
fig.show()